<a href="https://colab.research.google.com/github/Teja3993/my-first-repo/blob/main/R_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# make_r_ppts.py
!pip install python-pptx
# make_r_full_presentation.py
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Utility helpers
def set_font(paragraph, name='Calibri', size=18, bold=False, italic=False, color=(0,0,0)):
    for r in paragraph.runs:
        r.font.name = name
        r.font.size = Pt(size)
        r.font.bold = bold
        r.font.italic = italic
        r.font.color.rgb = RGBColor(*color)

def add_title_slide(prs, title, subtitle):
    slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    if subtitle:
        try:
            slide.placeholders[1].text = subtitle
        except Exception:
            pass
    return slide

def add_bullet_slide(prs, title, bullets, note=None):
    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    body = slide.shapes.placeholders[1].text_frame
    body.clear()
    for i, b in enumerate(bullets):
        if i == 0:
            p = body.paragraphs[0]
            p.text = b
        else:
            p = body.add_paragraph()
            p.text = b
        p.level = 0
        set_font(p, size=18)
    if note:
        try:
            notes = slide.notes_slide.notes_text_frame
            notes.text = note
        except Exception:
            pass
    return slide

def add_code_slide(prs, title, code, bullets=None, note=None):
    # Title + bullets on top, code box underneath
    slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title

    if bullets:
        body = slide.shapes.placeholders[1].text_frame
        body.clear()
        for i, b in enumerate(bullets):
            if i == 0:
                p = body.paragraphs[0]
                p.text = b
            else:
                p = body.add_paragraph()
                p.text = b
            p.level = 0
            set_font(p, size=16)

    # code textbox
    left = Inches(0.5)
    top = Inches(3.1)
    width = Inches(9)
    height = Inches(2.8)
    txBox = slide.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame
    tf.word_wrap = True
    p = tf.paragraphs[0]
    p.text = code
    # apply monospace and smaller font
    for run in p.runs:
        run.font.name = 'Courier New'
        run.font.size = Pt(12)
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT

    if note:
        try:
            notes = slide.notes_slide.notes_text_frame
            notes.text = note
        except Exception:
            pass

    return slide

def make_presentation(output="R_full_presentation_teaching_style.pptx"):
    prs = Presentation()
    # Title slide
    add_title_slide(prs,
                    "Introduction to R — Selected Topics (Teaching style)",
                    "Manipulating data | Simulation | Linear model | Data frame | Graphics\n(Python ↔ R comparisons included)")

    # Outline slide
    add_bullet_slide(prs, "Today's Roadmap",
                     ["1. Data frame",
                      "2. Manipulating data",
                      "3. Simulation",
                      "4. Linear model (lm)",
                      "5. Graphics (base & ggplot2)",
                      "Each topic: concept, syntax, examples, Python↔R notes"])

    # -------- Topic templates: each topic will have 5 slides ----------
    topics = []

    # 1) Data frame (5 slides)
    topics.append({
        "name": "Data frame",
        "slides": [
            {
                "title": "Data frames — Overview",
                "bullets": [
                    "Definition: rectangular (tabular) data structure in R — columns can have different types.",
                    "Use-case: typical data analysis, modeling input, reading/writing datasets."
                ],
                "note": "Emphasize: Data frames are the core tabular object in R (like pandas DataFrame in Python)."
            },
            {
                "title": "Creating & inspecting data.frames",
                "bullets": [
                    "Create: data.frame(), as.data.frame()",
                    "Inspect: str(), head(), summary(), dim(), names()"
                ],
                "code": (
                    "# create and inspect\n"
                    "df <- data.frame(id = 1:4,\n"
                    "                 age = c(21,25,30,28),\n"
                    "                 grp = c('A','B','A','B'))\n"
                    "str(df)\n"
                    "head(df)\n"
                ),
                "note": "Show how columns can be factors, characters, numeric. Mention stringsAsFactors historically."
            },
            {
                "title": "Reading and writing tabular data",
                "bullets": [
                    "Read: read.csv(), read.table(), readr::read_csv()",
                    "Write: write.csv(), write.table(), readr::write_csv()"
                ],
                "code": (
                    "# read and write\n"
                    "df2 <- read.csv('data.csv', stringsAsFactors = FALSE)\n"
                    "write.csv(df, 'out.csv', row.names = FALSE)\n"
                ),
                "note": "Recommend readr versions for speed: readr::read_csv()"
            },
            {
                "title": "Subsetting & selecting columns",
                "bullets": [
                    "Subset rows: df[rows, cols], subset()",
                    "Select columns: df$col, df[['col']], df[, 'col']"
                ],
                "code": (
                    "# examples\n"
                    "df_age_above25 <- df[df$age > 25, ]\n"
                    "ids <- df$id\n"
                    "subset(df, grp == 'A')\n"
                ),
                "note": "Compare to Python: df[df['age']>25] and df['col']"
            },
            {
                "title": "Data frame — R vs Python (quick)",
                "bullets": [
                    "R: data.frame(), use $ or [ , ] indexing; many base functions operate on columns.",
                    "Python: pandas.DataFrame, use df['col'] or df.col; chains with method chaining (.pipe, .assign).",
                    "Tip: When used to prepare data for models, convert factors as needed (factor())."
                ],
                "note": "Highlight factor (categorical) behavior in R vs pandas category dtype."
            }
        ]
    })

    # 2) Manipulating data (5 slides)
    topics.append({
        "name": "Manipulating data",
        "slides": [
            {
                "title": "Manipulating data — Overview",
                "bullets": [
                    "Operations: filter, select, arrange/sort, mutate (new columns), join/merge, reshape, aggregation.",
                    "Two common styles: base R and tidyverse (dplyr)."
                ],
                "note": "Show why dplyr pipelines are easier for step-by-step transforms."
            },
            {
                "title": "Key base-R functions",
                "bullets": [
                    "subset(), merge(), order(), apply()/lapply()/sapply(), aggregate(), reshape()"
                ],
                "code": (
                    "# base R aggregation\n"
                    "agg <- aggregate(mpg ~ cyl, data = mtcars, FUN = mean)\n"
                    "sorted <- mtcars[order(mtcars$mpg, decreasing = TRUE), ]\n"
                ),
                "note": "apply-family discussion: apply for matrices, lapply for lists, sapply to simplify."
            },
            {
                "title": "dplyr — idiomatic tidy data transforms",
                "bullets": [
                    "Common verbs: filter(), select(), mutate(), arrange(), group_by(), summarise(), join functions."
                ],
                "code": (
                    "# dplyr pipeline\n"
                    "library(dplyr)\n"
                    "mtcars %>%\n"
                    "  filter(cyl == 6) %>%\n"
                    "  group_by(gear) %>%\n"
                    "  summarise(mean_mpg = mean(mpg))\n"
                ),
                "note": "Install tidyverse: install.packages('tidyverse')"
            },
            {
                "title": "Reshaping & merging",
                "bullets": [
                    "reshape2/tidyr for wide↔long: gather()/spread() or pivot_longer()/pivot_wider().",
                    "Merges: merge(x,y,by=...), dplyr::left_join(), inner_join(), full_join()."
                ],
                "code": (
                    "# tidyr example\n"
                    "library(tidyr)\n"
                    "wide <- data.frame(id=1:3, a=c(1,2,3), b=c(4,5,6))\n"
                    "long <- pivot_longer(wide, cols = c(a,b), names_to = 'var', values_to = 'val')\n"
                ),
                "note": "Explain keys when joining; check join result size to avoid unintended Cartesian joins."
            },
            {
                "title": "Manipulating — R vs Python tips",
                "bullets": [
                    "dplyr pipelines (R) ≈ method chaining with pandas (Python).",
                    "Use vectorized ops; avoid for-loops for row-wise operations when possible.",
                    "Always validate shapes after joins and aggregations."
                ],
                "note": "Show short mapping: filter -> df[df['col']==x], group_by+summarise -> groupby().agg()"
            }
        ]
    })

    # 3) Simulation (5 slides)
    topics.append({
        "name": "Simulation",
        "slides": [
            {
                "title": "Simulation — Why and when",
                "bullets": [
                    "Generate synthetic data to test methods, estimate sampling variability, teach probability concepts.",
                    "Common tasks: sampling, Monte Carlo experiments, bootstrapping."
                ],
                "note": "Mention reproducibility with set.seed()."
            },
            {
                "title": "Basic random generation functions",
                "bullets": [
                    "rnorm(n, mean, sd), runif(n, min, max), rbinom(n, size, prob), rexp(), rpois().",
                    "d* versions give densities (dnorm), p* give cumulative (pnorm)."
                ],
                "code": (
                    "# basic generators\n"
                    "set.seed(42)\n"
                    "x <- rnorm(1000, mean=5, sd=2)\n"
                    "u <- runif(100)\n"
                    "y <- rbinom(50, size=1, prob=0.3)\n"
                ),
                "note": "Show histograms and summary statistics to inspect distributions."
            },
            {
                "title": "Monte Carlo example",
                "bullets": [
                    "Estimate probability or expectation by repeated simulation and averaging.",
                ],
                "code": (
                    "# Monte Carlo: estimate P(mean(sample)>threshold)\n"
                    "set.seed(1)\n"
                    "B <- 5000\n"
                    "res <- replicate(B, mean(rnorm(30, mean=0, sd=1)) > 0.2)\n"
                    "mean(res)\n"
                ),
                "note": "Discuss variance of estimator and increasing B to reduce simulation error."
            },
            {
                "title": "Bootstrap example (resampling)",
                "bullets": [
                    "Bootstrap: resample with replacement to estimate standard errors/confidence intervals."
                ],
                "code": (
                    "# bootstrap of mean\n"
                    "set.seed(2)\n"
                    "x <- rnorm(50)\n"
                    "B <- 2000\n"
                    "bstat <- replicate(B, mean(sample(x, replace = TRUE)))\n"
                    "quantile(bstat, c(0.025, 0.975))\n"
                ),
                "note": "Mention boot package and caution: bootstrap assumptions."
            },
            {
                "title": "Simulation — R vs Python",
                "bullets": [
                    "R base random: rnorm/runif/rbinom. Python: numpy.random.normal, random.sample, scipy.stats.",
                    "Set seeds: set.seed() (R) vs numpy.random.seed() (Python).",
                    "Use vectorized sampling for speed; use replicate() or loops depending on complexity."
                ],
                "note": "If heavy simulation needed, consider Rcpp (C++) or parallel packages."
            }
        ]
    })

    # 4) Linear model (5 slides)
    topics.append({
        "name": "Linear model",
        "slides": [
            {
                "title": "Linear models — Concept",
                "bullets": [
                    "Models continuous response as linear combination of predictors + error.",
                    "Used for inference, prediction, and understanding relationships."
                ],
                "note": "Introduce assumptions briefly: linearity, homoscedasticity, independence, normality of errors."
            },
            {
                "title": "Fitting models with lm()",
                "bullets": [
                    "Syntax: lm(formula = y ~ x1 + x2, data = ...).",
                    "Examine: summary(), coef(), confint(), residuals(), predict()."
                ],
                "code": (
                    "# simple linear regression\n"
                    "model <- lm(mpg ~ wt + hp, data = mtcars)\n"
                    "summary(model)\n"
                    "coef(model)\n"
                ),
                "note": "Explain formula interface and how to include interactions: y ~ x1 * x2"
            },
            {
                "title": "Diagnostics & validation",
                "bullets": [
                    "Residual plots, qqnorm(residuals(model)), plot(model) to check assumptions.",
                    "Use broom::tidy() for neat model summaries, or car::vif() for multicollinearity checks."
                ],
                "code": (
                    "# diagnostics\n"
                    "par(mfrow = c(2,2))\n"
                    "plot(model)\n"
                    "par(mfrow = c(1,1))\n"
                ),
                "note": "Interpret R-squared, adjusted R-squared, p-values, and coefficients with domain context."
            },
            {
                "title": "Extensions & prediction",
                "bullets": [
                    "Generalized linear models: glm() for logistic, Poisson, etc.",
                    "Use predict(model, newdata, interval='confidence') for predicted values."
                ],
                "code": (
                    "# logistic example\n"
                    "glm_mod <- glm(vs ~ wt + mpg, data = mtcars, family = binomial)\n"
                    "summary(glm_mod)\n"
                ),
                "note": "Point out difference: lm() vs glm() families and link functions."
            },
            {
                "title": "Linear model — R vs Python",
                "bullets": [
                    "R: lm() + formula interface is very expressive and concise.",
                    "Python: statsmodels.formula.api.ols(formula, data).fit() gives a similar API; sklearn focuses on prediction pipelines.",
                    "Tip: Use statsmodels for inference, sklearn for pipelines/production."
                ],
                "note": "Show small example mapping if needed during talk."
            }
        ]
    })

    # 5) Graphics in R (5 slides)
    topics.append({
        "name": "Graphics",
        "slides": [
            {
                "title": "Graphics in R — Overview",
                "bullets": [
                    "Two main ecosystems: base R graphics and ggplot2 (grammar of graphics).",
                    "Base: quick plots; ggplot2: layered, declarative, great for publication-quality figures."
                ],
                "note": "Mention lattice as another option but focus on base + ggplot2."
            },
            {
                "title": "Base plotting quick reference",
                "bullets": [
                    "plot(), hist(), boxplot(), barplot(), lines(), points().",
                    "Save to files: png('file.png'); pdf('file.pdf'); dev.off()"
                ],
                "code": (
                    "# base plot\n"
                    "plot(mtcars$wt, mtcars$mpg, pch = 19, main = 'MPG vs Weight')\n"
                    "abline(lm(mpg ~ wt, data = mtcars))\n"
                ),
                "note": "Base is useful for quick exploratory plots and overlays."
            },
            {
                "title": "ggplot2 — grammar of graphics",
                "bullets": [
                    "Core: ggplot(data, aes(x, y)) + geom_point() + geom_smooth(method='lm') + labs().",
                    "Facets and themes offer layout and styling control."
                ],
                "code": (
                    "library(ggplot2)\n"
                    "ggplot(mtcars, aes(x = wt, y = mpg)) +\n"
                    "  geom_point() +\n"
                    "  geom_smooth(method = 'lm') +\n"
                    "  ggtitle('MPG vs Weight')\n"
                ),
                "note": "ggplot2 is part of tidyverse and integrates well with dplyr pipelines."
            },
            {
                "title": "Saving & customizing plots",
                "bullets": [
                    "ggsave('plot.png', width=6, height=4) for ggplot2. For base: png()/dev.off().",
                    "Customize labels, legends, color scales, themes (theme_minimal(), theme_bw())."
                ],
                "code": (
                    "# save ggplot\n"
                    "p <- ggplot(mtcars, aes(wt, mpg)) + geom_point()\n"
                    "ggsave('myplot.png', plot = p, width = 6, height = 4)\n"
                ),
                "note": "Discuss vector vs raster differences when saving for publications."
            },
            {
                "title": "Graphics — R vs Python",
                "bullets": [
                    "R: ggplot2 provides a consistent grammar of graphics (like plotnine in Python).",
                    "Python: matplotlib (low-level), seaborn (higher-level), plotnine (ggplot-like).",
                    "Tip: If you know ggplot in R, try plotnine in Python for similar syntax."
                ],
                "note": "Encourage learners to practice porting a plot from R to Python to solidify understanding."
            }
        ]
    })

    # Add all slides from topics in order
    for top in topics:
        for s in top['slides']:
            title = s.get('title', top['name'])
            bullets = s.get('bullets', None)
            code = s.get('code', None)
            note = s.get('note', None)
            if code:
                add_code_slide(prs, title, code, bullets=bullets, note=note)
            else:
                add_bullet_slide(prs, title, bullets or [], note=note)

    # Final slide: Resources & references
    add_bullet_slide(prs, "Resources & Next Steps",
                     [
                         "R manuals and CRAN documentation (cran.r-project.org).",
                         "Tidyverse: https://www.tidyverse.org/ (dplyr, ggplot2, tidyr, readr).",
                         "Recommended practice: translate Python snippets you know into R to learn quickly.",
                         "Try examples in RStudio; keep a reproducible script/notebook for each example."
                     ],
                     note="If you want, I can embed short excerpts from your uploaded slides into specific slides (verbatim) — tell me which file and slide to include.")

    # Save
    prs.save(output)
    print("Saved presentation to:", output)

if __name__ == "__main__":
    make_presentation()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.0 MB/s eta 0:00:00
Saved presentation to: R_full_presentation_teaching_style.pptx


In [2]:
# make_r_full_presentation_styled_full.py
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# ==========================================================
# UTILITY FUNCTIONS (for colors, fonts, layout, backgrounds)
# ==========================================================

def set_font(paragraph, name='Calibri', size=22, bold=False, italic=False, color=(30, 30, 30)):
    for r in paragraph.runs:
        r.font.name = name
        r.font.size = Pt(size)
        r.font.bold = bold
        r.font.italic = italic
        r.font.color.rgb = RGBColor(*color)


def add_background(slide, color=(230, 240, 255)):
    """Add full-slide color background."""
    shape = slide.shapes.add_shape(
        1, Inches(0), Inches(0), Inches(13.33), Inches(7.5)
    )
    shape.fill.solid()
    shape.fill.fore_color.rgb = RGBColor(*color)
    shape.line.fill.background()


def add_title_slide(prs, title, subtitle):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    add_background(slide)
    title_box = slide.shapes.add_textbox(Inches(1), Inches(2.5), Inches(11), Inches(2))
    tf = title_box.text_frame
    p = tf.add_paragraph()
    p.text = title
    p.font.size = Pt(46)
    p.font.bold = True
    p.font.color.rgb = RGBColor(10, 60, 130)
    tf.word_wrap = True
    if subtitle:
        sub_box = slide.shapes.add_textbox(Inches(1), Inches(4.2), Inches(11), Inches(1))
        tf2 = sub_box.text_frame
        p2 = tf2.add_paragraph()
        p2.text = subtitle
        p2.font.size = Pt(24)
        p2.font.color.rgb = RGBColor(70, 70, 70)
    return slide


def add_bullet_slide(prs, title, bullets, note=None):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    add_background(slide)
    # Title bar
    tbox = slide.shapes.add_textbox(Inches(0.8), Inches(0.5), Inches(11), Inches(1.2))
    tf_title = tbox.text_frame
    p = tf_title.add_paragraph()
    p.text = title
    p.font.size = Pt(40)
    p.font.bold = True
    p.font.color.rgb = RGBColor(230, 120, 30)

    # Bullets
    content_box = slide.shapes.add_textbox(Inches(1), Inches(2), Inches(11), Inches(4.5))
    tf = content_box.text_frame
    tf.word_wrap = True
    for b in bullets:
        p = tf.add_paragraph()
        p.text = b
        set_font(p, size=22)
    if note:
        slide.notes_slide.notes_text_frame.text = note
    return slide


def add_code_slide(prs, title, code, bullets=None, note=None):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    add_background(slide)

    # Title
    tbox = slide.shapes.add_textbox(Inches(0.8), Inches(0.5), Inches(11), Inches(1.2))
    tf_title = tbox.text_frame
    p = tf_title.add_paragraph()
    p.text = title
    p.font.size = Pt(38)
    p.font.bold = True
    p.font.color.rgb = RGBColor(230, 120, 30)

    # Bullets
    if bullets:
        bullet_box = slide.shapes.add_textbox(Inches(1.0), Inches(1.8), Inches(11), Inches(2))
        tf_bul = bullet_box.text_frame
        for b in bullets:
            p = tf_bul.add_paragraph()
            p.text = b
            set_font(p, size=20)

    # Code box
    code_box = slide.shapes.add_textbox(Inches(0.9), Inches(3.8), Inches(11.3), Inches(3))
    code_box.fill.solid()
    code_box.fill.fore_color.rgb = RGBColor(245, 245, 245)
    tf_code = code_box.text_frame
    tf_code.word_wrap = True
    p = tf_code.add_paragraph()
    p.text = code
    for r in p.runs:
        r.font.name = "Consolas"
        r.font.size = Pt(16)
        r.font.color.rgb = RGBColor(20, 20, 20)
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT

    if note:
        slide.notes_slide.notes_text_frame.text = note
    return slide


# ==========================================================
# MAIN PRESENTATION LOGIC
# ==========================================================

def make_presentation(output="R_full_presentation_styled_full.pptx"):
    prs = Presentation()

    # Title & outline
    add_title_slide(
        prs,
        "Introduction to R — Selected Topics (Teaching Style)",
        "Manipulating data | Simulation | Linear model | Data frame | Graphics\n(Python ↔ R comparisons included)"
    )

    add_bullet_slide(
        prs,
        "Today's Roadmap",
        [
            "1. Data frame",
            "2. Manipulating data",
            "3. Simulation",
            "4. Linear model (lm)",
            "5. Graphics (base & ggplot2)",
            "Each topic: concept, syntax, examples, Python↔R notes"
        ]
    )

    # ========================
    # TOPICS
    # ========================

    topics = [
        # Data Frame
        {
            "title": "Data Frames — Overview",
            "bullets": [
                "Rectangular tabular data structures in R where columns can differ in type.",
                "Used for most data analysis tasks — similar to pandas DataFrame.",
                "Support row and column indexing, metadata, and modeling compatibility."
            ]
        },
        {
            "title": "Creating & Inspecting Data Frames",
            "bullets": ["Create: data.frame(), as.data.frame()", "Inspect: str(), head(), summary(), names()"],
            "code": "df <- data.frame(id=1:4, age=c(21,25,30,28), group=c('A','B','A','B'))\nstr(df)\nhead(df)"
        },
        {
            "title": "Reading & Writing Data",
            "bullets": ["Read CSVs: read.csv(), readr::read_csv()", "Write: write.csv(), readr::write_csv()"],
            "code": "df2 <- read.csv('data.csv')\nwrite.csv(df2, 'output.csv', row.names=FALSE)"
        },
        {
            "title": "Subsetting & Selecting Columns",
            "bullets": ["Subset rows: df[rows, cols]", "Select: df$col, subset(df, col>value)"],
            "code": "subset(df, age > 25)\nids <- df$id"
        },
        {
            "title": "Data Frame — R vs Python",
            "bullets": [
                "R: uses $ or [ , ] for access; Python uses df['col'].",
                "Factor variables in R ≈ categorical dtype in pandas.",
                "Both support summarization and column operations."
            ]
        },

        # Manipulating Data
        {
            "title": "Manipulating Data — Overview",
            "bullets": [
                "Includes filtering, selecting, sorting, joining, reshaping, aggregation.",
                "Done with base R functions or tidyverse's dplyr."
            ]
        },
        {
            "title": "Key Base R Functions",
            "bullets": ["subset(), merge(), order(), apply(), aggregate()"],
            "code": "agg <- aggregate(mpg ~ cyl, data=mtcars, FUN=mean)\nsorted <- mtcars[order(mtcars$mpg), ]"
        },
        {
            "title": "Using dplyr for Clean Pipelines",
            "bullets": ["filter(), select(), mutate(), arrange(), group_by(), summarise()."],
            "code": "library(dplyr)\nmtcars %>% filter(cyl==6) %>% group_by(gear) %>% summarise(mean_mpg=mean(mpg))"
        },
        {
            "title": "Reshaping & Joining Data",
            "bullets": ["Use tidyr pivot_longer/pivot_wider for reshaping.", "Use left_join(), inner_join() for merges."],
            "code": "library(tidyr)\nlong <- pivot_longer(df, cols=c(a,b), names_to='var', values_to='val')"
        },
        {
            "title": "Manipulating Data — R vs Python",
            "bullets": [
                "R dplyr pipelines ≈ pandas method chaining.",
                "Avoid row-wise loops — prefer vectorized ops.",
                "Always validate data after merges or aggregations."
            ]
        },

        # Simulation
        {
            "title": "Simulation — Why & When",
            "bullets": [
                "Used for generating synthetic data, testing models, or teaching statistics.",
                "Core idea: random sampling and repetition (Monte Carlo)."
            ]
        },
        {
            "title": "Random Generation Functions",
            "bullets": ["rnorm(), runif(), rbinom(), rexp(), rpois().", "Set seed for reproducibility."],
            "code": "set.seed(42)\nx <- rnorm(100, mean=0, sd=1)\nhist(x)"
        },
        {
            "title": "Monte Carlo Example",
            "bullets": ["Estimate probability by repeated sampling."],
            "code": "set.seed(1)\nmean(replicate(10000, mean(rnorm(30))>0.2))"
        },
        {
            "title": "Bootstrap Resampling",
            "bullets": ["Estimate standard errors/confidence intervals by resampling."],
            "code": "x <- rnorm(50)\nboot <- replicate(2000, mean(sample(x, replace=TRUE)))\nquantile(boot, c(0.025,0.975))"
        },
        {
            "title": "Simulation — R vs Python",
            "bullets": ["R: rnorm/runif/rbinom; Python: numpy.random.", "Set seed: set.seed() vs np.random.seed()."]
        },

        # Linear Model
        {
            "title": "Linear Models — Concept",
            "bullets": [
                "Linear regression models numeric response as function of predictors.",
                "Used for prediction and inference under linearity assumptions."
            ]
        },
        {
            "title": "Fitting Models with lm()",
            "bullets": ["lm(y ~ x1 + x2, data=df)", "Check summary(), coef(), residuals()."],
            "code": "model <- lm(mpg ~ wt + hp, data=mtcars)\nsummary(model)"
        },
        {
            "title": "Diagnostics & Validation",
            "bullets": ["Residual plots, QQ plots, VIF for multicollinearity."],
            "code": "par(mfrow=c(2,2))\nplot(model)"
        },
        {
            "title": "Extensions & Prediction",
            "bullets": ["Generalized models: glm() for logistic, Poisson.", "predict(model, newdata)."],
            "code": "glm_mod <- glm(vs ~ wt + mpg, data=mtcars, family=binomial)\nsummary(glm_mod)"
        },
        {
            "title": "Linear Models — R vs Python",
            "bullets": [
                "R: lm() concise for inference.",
                "Python: statsmodels for inference, sklearn for ML pipelines."
            ]
        },

        # Graphics
        {
            "title": "Graphics in R — Overview",
            "bullets": [
                "Base R: simple quick plots.",
                "ggplot2: layered grammar of graphics for publication-quality visuals."
            ]
        },
        {
            "title": "Base Plotting",
            "bullets": ["plot(), hist(), boxplot(), barplot().", "Save: png(), pdf(), dev.off()."],
            "code": "plot(mtcars$wt, mtcars$mpg)\nabline(lm(mpg~wt, data=mtcars))"
        },
        {
            "title": "ggplot2 — Grammar of Graphics",
            "bullets": ["ggplot(data,aes()) + geom_*()", "Add layers, titles, and themes."],
            "code": "library(ggplot2)\nggplot(mtcars,aes(wt,mpg))+geom_point()+geom_smooth(method='lm')"
        },
        {
            "title": "Saving & Customizing Plots",
            "bullets": ["Use ggsave('plot.png') for ggplot.", "theme_minimal() for clean visuals."],
            "code": "p <- ggplot(mtcars,aes(wt,mpg))+geom_point()\nggsave('plot.png',plot=p)"
        },
        {
            "title": "Graphics — R vs Python",
            "bullets": [
                "ggplot2 in R ≈ plotnine in Python.",
                "Python: matplotlib (low-level), seaborn (high-level)."
            ]
        }
    ]

    # Create slides
    for t in topics:
        if "code" in t:
            add_code_slide(prs, t["title"], t["code"], bullets=t["bullets"])
        else:
            add_bullet_slide(prs, t["title"], t["bullets"])

    # Final slide
    add_bullet_slide(
        prs,
        "Resources & Next Steps",
        [
            "CRAN R manuals: https://cran.r-project.org/manuals.html",
            "Tidyverse: https://www.tidyverse.org/",
            "Try rewriting Python code examples in R.",
            "Explore RStudio notebooks for reproducibility."
        ]
    )

    prs.save(output)
    print(f"✅ Saved: {output}")


if __name__ == "__main__":
    make_presentation()


✅ Saved: R_full_presentation_styled_full.pptx
